In [ ]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

In [ ]:
from openai import OpenAI

client = OpenAI() # defautls to getting the key using os.environ.get("OPENAI_API_KEY")

In [ ]:
# Upload LinkedIn profile to OpenAI
profile = client.files.create(
  file=open("LinkedIn_profile.json", "rb"),
  purpose="assistants"
)
print(profile)

In [ ]:
# read in contents of job description
with open("job_description.json", "r") as f:
    job_description = f.read()
print(job_description)

In [ ]:
# Attach the example linkedIn profile to this thread
# OpenAI automatically creates a vector store for the file that expires in 7 days
thread = client.beta.threads.create(
    tool_resources={
        'file_search': {
            'vector_stores': [
                {
                    'file_ids': [profile.id]
                }
            ]
        }
    },
    messages=[
        {
            'role': 'user',
            'content': job_description
        }
    ]
)

print(thread)

In [ ]:
import os

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=os.environ.get("ASSISTANT_ID")
)

print(run)

In [ ]:
# Retreive all run steps
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

# Find message creation text step and extract content
text_creation_step = run_steps.data[0]
message_id = text_creation_step.step_details.message_creation.message_id

message = client.beta.threads.messages.retrieve(
  message_id=message_id,
  thread_id=thread.id,
)

message_text = message.content[0].text.value
print(message_text)

In [ ]:
import json

# Extract the JSON resume from the message content
start = message_text.index("```json") + 7
end = message_text.index("```", start)

json_resume = json.loads(message_text[start:end])
print(json_resume)

with open('custom_resume.json', 'w') as output_file:
    json.dump(json_resume, output_file)